# Goal 1
Refactor the Polygon class so that all the calculated properties are lazy properties, i.e. they should still be calculated properties, but they should not have to get recalculated more than once (since we made our Polygon class "immutable").


## Approach 1

In [19]:
import math


def memoize(fn):
    """
    Decorator for memoization
    :param fn: Function on which memoization to be implemented
    :return: Closure (inner)
    """
    from functools import wraps
    cache = dict()

    @wraps(fn)
    def inner(self=None):
        """
        Closure to store and update cache
        :param args: arguments passed to the function
        :return: output of the function from cache if output is present in cache
        """
        if fn.__name__ not in cache:
            cache[fn.__name__] = fn(self)
        return cache[fn.__name__]
    return inner


class Polygon1(object):
    def __init__(self, n, R):
        if n < 3:
            raise ValueError('Polygon must have at least 3 vertices.')
        self._n = n
        self._R = R
        self.cache = dict()
        
    def __repr__(self):
        return f'Polygon(n={self._n}, R={self._R})'
    
    @property
    @memoize
    def count_vertices(self):
        return self._n
    
    @property
    @memoize
    def count_edges(self):
        return self._n
    
    @property
    @memoize
    def circumradius(self):
        return self._R
    
    @property
    @memoize
    def interior_angle(self):
        return (self._n - 2) * 180 / self._n

    @property
    @memoize
    def side_length(self):
        return 2 * self._R * math.sin(math.pi / self._n)
    
    @property
    @memoize
    def apothem(self):
        return self._R * math.cos(math.pi / self._n)
    
    @property
    @memoize
    def area(self):
        return self._n / 2 * self.side_length * self.apothem
    
    @property
    @memoize
    def perimeter(self):
        return self._n * self.side_length
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.count_edges == other.count_edges 
                    and self.circumradius == other.circumradius)
        else:
            return NotImplemented
        
    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self.count_vertices > other.count_vertices
        else:
            return NotImplemented

In [20]:
p = Polygon1(4, 5)

In [21]:
print(f"Vertices: {p.count_vertices}")
print(f"Edges: {p.count_edges}")
print(f"Circumradius: {p.circumradius}")
print(f"Interior Angle: {p.interior_angle}")
print(f"Side length: {p.side_length}")
print(f"Apothem: {p.apothem}")
print(f"Area: {p.area}")
print(f"Perimeter: {p.perimeter}")

Vertices: 4
Edges: 4
Circumradius: 5
Interior Angle: 90.0
Side length: 7.071067811865475
Apothem: 3.5355339059327378
Area: 50.0
Perimeter: 28.2842712474619


In [23]:
p = Polygon1(5, 5)

print(f"Vertices: {p.count_vertices}")
print(f"Edges: {p.count_edges}")
print(f"Circumradius: {p.circumradius}")
print(f"Interior Angle: {p.interior_angle}")
print(f"Side length: {p.side_length}")
print(f"Apothem: {p.apothem}")
print(f"Area: {p.area}")
print(f"Perimeter: {p.perimeter}")

Vertices: 4
Edges: 4
Circumradius: 5
Interior Angle: 90.0
Side length: 7.071067811865475
Apothem: 3.5355339059327378
Area: 50.0
Perimeter: 28.2842712474619


Here, even after creating new instance the data returned is from the old instance due to latching to cache  
This breaks the functionality of the class


## Appraoch 2

In [24]:
class lazy(object):
    def __init__(self, func):
        self.func = func
 
    def __get__(self, instance, cls):
        val = self.func(instance)
        setattr(instance, self.func.__name__, val)
        return val


class LazyPolygon:
    def __init__(self, n, R):
        if n < 3:
            raise ValueError('Polygon must have at least 3 vertices.')
        self._n = n
        self._R = R
        self.cache = dict()
        
    def __repr__(self):
        return f'Polygon(n={self._n}, R={self._R})'
    
    @lazy
    def count_vertices(self):
        return self._n
    
    @lazy
    def count_edges(self):
        return self._n
    
    @lazy
    def circumradius(self):
        return self._R
    
    @lazy
    def interior_angle(self):
        return (self._n - 2) * 180 / self._n

    @lazy
    def side_length(self):
        return 2 * self._R * math.sin(math.pi / self._n)
    
    @lazy
    def apothem(self):
        return self._R * math.cos(math.pi / self._n)
    
    @lazy
    def area(self):
        return self._n / 2 * self.side_length * self.apothem
    
    @lazy
    def perimeter(self):
        return self._n * self.side_length
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.count_edges == other.count_edges 
                    and self.circumradius == other.circumradius)
        else:
            return NotImplemented
        
    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self.count_vertices > other.count_vertices
        else:
            return NotImplemented

In [25]:
p2 = LazyPolygon(3, 5)

print(f"Vertices: {p2.count_vertices}")
print(f"Edges: {p2.count_edges}")
print(f"Circumradius: {p2.circumradius}")
print(f"Interior Angle: {p2.interior_angle}")
print(f"Side length: {p2.side_length}")
print(f"Apothem: {p2.apothem}")
print(f"Area: {p2.area}")
print(f"Perimeter: {p2.perimeter}")

Vertices: 3
Edges: 3
Circumradius: 5
Interior Angle: 60.0
Side length: 8.660254037844386
Apothem: 2.5000000000000004
Area: 32.47595264191645
Perimeter: 25.980762113533157


In [26]:
p2.count_vertices = 5

print(f"Vertices: {p2.count_vertices}")
print(f"Edges: {p2.count_edges}")
print(f"Circumradius: {p2.circumradius}")
print(f"Interior Angle: {p2.interior_angle}")
print(f"Side length: {p2.side_length}")
print(f"Apothem: {p2.apothem}")
print(f"Area: {p2.area}")
print(f"Perimeter: {p2.perimeter}")

Vertices: 5
Edges: 3
Circumradius: 5
Interior Angle: 60.0
Side length: 8.660254037844386
Apothem: 2.5000000000000004
Area: 32.47595264191645
Perimeter: 25.980762113533157


Here, the class is still mutable and the other properties are still cached so the output values are not related to valid regular convex polygon

## Approach 3

In [27]:
import math


def lazy_property(func):
    """
    Decorator function for lazy calculations of the property
    :param func: method to be decorator
    :return: closure
    """
    attr_name = "_lazy_" + func.__name__
 
    @property
    def _lazy_property(self):
        """
        Function to calculate the property if not already done
        :param self: instance
        :return: value of the property
        """
        if not hasattr(self, attr_name):
            setattr(self, attr_name, func(self))
        return getattr(self, attr_name)
    return _lazy_property


class Polygon:
    """
    Class for lazy properties
    """
    def __init__(self, n, circumradius):
        """
        Constructor
        """
        if n < 3:
            raise ValueError('Polygon must have at least 3 vertices.')
        self._n = n
        self._R = circumradius
        self.cache = dict()
        
    def __repr__(self):
        """
        Function to print the details of class instance
        """
        return f'Polygon(n={self._n}, R={self._R})'
    
    @lazy_property
    def count_vertices(self):
        """
        Property to return number of vertices of the regular polygon
        """
        return self._n
    
    @lazy_property
    def count_edges(self):
        """
        Property to return number of edges of the regular polygon
        """
        return self._n
    
    @lazy_property
    def circumradius(self):
        """
        Property to return circumradius of the regular polygon
        """
        return self._R
    
    @lazy_property
    def interior_angle(self):
        """
        Property to return interior angle of the regular polygon
        """
        return (self._n - 2) * 180 / self._n

    @lazy_property
    def side_length(self):
        """
        Property to return length of each side of the regular polygon
        """
        return 2 * self._R * math.sin(math.pi / self._n)
    
    @lazy_property
    def apothem(self):
        """
        Property to return perpendicular distance from center of circle to side of the regular polygon
        """
        return self._R * math.cos(math.pi / self._n)
    
    @lazy_property
    def area(self):
        """
        Property to return area of the regular polygon
        """
        return self._n / 2 * self.side_length * self.apothem
    
    @lazy_property
    def perimeter(self):
        """
        Property to return perimeter of the regular polygon
        """
        return self._n * self.side_length
    
    def __eq__(self, other):
        """
        Property to implement equality operator for multiple regular polygon
        """
        if isinstance(other, self.__class__):
            return (self.count_edges == other.count_edges 
                    and self.circumradius == other.circumradius)
        else:
            return NotImplemented
        
    def __gt__(self, other):
        """
        Property to implement greater than operator for multiple regular polygon
        """
        if isinstance(other, self.__class__):
            return self.count_vertices > other.count_vertices
        else:
            return NotImplemented

In [28]:
# Create an instance and test properties
p3 = Polygon(3, 5)

print(f"Vertices: {p3.count_vertices}")
print(f"Edges: {p3.count_edges}")
print(f"Circumradius: {p3.circumradius}")
print(f"Interior Angle: {p3.interior_angle}")
print(f"Side length: {p3.side_length}")
print(f"Apothem: {p3.apothem}")
print(f"Area: {p3.area}")
print(f"Perimeter: {p3.perimeter}")

Vertices: 3
Edges: 3
Circumradius: 5
Interior Angle: 60.0
Side length: 8.660254037844386
Apothem: 2.5000000000000004
Area: 32.47595264191645
Perimeter: 25.980762113533157


In [44]:
# Check if the class is immutable
p3.count_vertices = 5

AttributeError: ignored

In [45]:
# Check if output is correct and no latching on the cache
p3 = Polygon(4, 5)

print(f"Vertices: {p3.count_vertices}")
print(f"Edges: {p3.count_edges}")
print(f"Circumradius: {p3.circumradius}")
print(f"Interior Angle: {p3.interior_angle}")
print(f"Side length: {p3.side_length}")
print(f"Apothem: {p3.apothem}")
print(f"Area: {p3.area}")
print(f"Perimeter: {p3.perimeter}")

Vertices: 4
Edges: 4
Circumradius: 5
Interior Angle: 90.0
Side length: 7.071067811865475
Apothem: 3.5355339059327378
Area: 50.0
Perimeter: 28.2842712474619


---
---

### Goal 2
Refactor the Polygons (sequence) type, into an iterable. Make sure also that the elements in the iterator are computed lazily - i.e. you can no longer use a list as an underlying storage mechanism for your polygons.

You'll need to implement both an iterable, and an iterator.

In [31]:
class Polygons:
    """
    Iterable for regular polygons
    """
    def __init__(self, circumradius):
        """
        Constructor
        :param circumradius: radius of a circle in which the polygons are inscribed in
        """
        if circumradius < 0:
            raise ValueError('R must be greater than 0')
        self._m = 2
        self._R = circumradius
        self._instance = None

    def __iter__(self):
        """
        Method to make this class Iterable
        :return: Instance of the Iterator class
        """
        self._instance = self.PolygonIterator(self)
        return self._instance

    class PolygonIterator:
        """
        Iterator class for the Iterable
        """
        def __init__(self, obj):
            """
            Constructor
            """
            self._obj = obj
            self._ratio_data = dict()
            self.collection = []

        def __iter__(self):
            """
            Method to make this class Iterator
            """
            return self

        def __next__(self):
            """
            Method to iterate over the values of iterator
            """
            # Increment the number of edges
            self._obj._m += 1

            # Create a polygon with number of edges and circumradius
            polygon = Polygon(self._obj._m, self._obj._R)

            # Calculate the ratio using the porperties of the polygon
            ratio = polygon.area / polygon.perimeter

            # Keep a record of edges and ratio
            if self._obj._m not in self._ratio_data:
                self._ratio_data[self._obj._m] = ratio

            # Append all the polygon instances
            self.collection.append(polygon)

            return polygon

        @property
        def ratio_data(self):
            """
            Property to access ratio of area to perimeter of all the polygon
            """
            return self._ratio_data

        @property
        def get_collection(self):
            """
            Property to access instances of all the polygons
            """
            return self.collection
        
    def __len__(self):
        """
        Method to print the number of polygons in an instance
        """
        return len(self._instance.ratio_data)
    
    def __repr__(self):
        """
        Method to print information of collections
        """ 
        return f'Collection of regular Polygons from edges 3 to {len(self._instance.ratio_data) + 2}'
    
    def __getitem__(self, index):
        """
        Method to access the polygons using slicing and indexing
        """
        if isinstance(index, int):
            if index < 0:
                index = len(self._instance.ratio_data) + index
            if index < 0 or index > len(self._instance.ratio_data):
                raise IndexError
            else:
                return self._instance.get_collection[index]
        else:
            start, stop, step = index.indices(len(self._instance.ratio_data))
            rng = range(start, stop, step)
            return [self._instance.get_collection[index] for index in rng]
    
    @property
    def max_efficiency_polygon(self):
        """
        Property to access the number of edges of a polygon with maximum area to perimeter ratio
        """
        sorted_polygons = sorted(self._instance.ratio_data, reverse=True)
        return sorted_polygons[0]

In [36]:
# Access the Lazy Iterator
polys = Polygons(circumradius=5)
polys_iter = iter(polys)

In [37]:
# Test Iterator functionality
for _ in range(20):
    polygon = next(polys_iter)
    print(f"Edges: {polygon.count_edges}")

Edges: 3
Edges: 4
Edges: 5
Edges: 6
Edges: 7
Edges: 8
Edges: 9
Edges: 10
Edges: 11
Edges: 12
Edges: 13
Edges: 14
Edges: 15
Edges: 16
Edges: 17
Edges: 18
Edges: 19
Edges: 20
Edges: 21
Edges: 22


In [39]:
# Test __repr__ functionality
print(f"Repr data: {polys}")

Repr data: Collection of regular Polygons from edges 3 to 22


In [40]:
# Test __len__ functionality
print(f"Length: {len(polys)}")

Length: 20


In [41]:
# Test Indexing functionality
print(f"Indexing: {polys[::-1]}")

Indexing: [Polygon(n=22, R=5), Polygon(n=21, R=5), Polygon(n=20, R=5), Polygon(n=19, R=5), Polygon(n=18, R=5), Polygon(n=17, R=5), Polygon(n=16, R=5), Polygon(n=15, R=5), Polygon(n=14, R=5), Polygon(n=13, R=5), Polygon(n=12, R=5), Polygon(n=11, R=5), Polygon(n=10, R=5), Polygon(n=9, R=5), Polygon(n=8, R=5), Polygon(n=7, R=5), Polygon(n=6, R=5), Polygon(n=5, R=5), Polygon(n=4, R=5), Polygon(n=3, R=5)]


In [42]:
# Test Slicing functionality
print(f"Slicing: {polys[2:5:1]}")

Slicing: [Polygon(n=5, R=5), Polygon(n=6, R=5), Polygon(n=7, R=5)]


In [43]:
# Test maximum efficiency functionality
print(f"Max efficiency: {polys.max_efficiency_polygon}")

Max efficiency: 22
